本次实验的数据集与Lab3一致，任务都是识别手写数字0-9。不过在本次实验中，我们需要采用**神经网络**来实现

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [2]:
data = loadmat('ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

由于我们后续需要经常用到样本集，所以我们定义一个别名

In [3]:
X = data['X']
y = data['y']
X.shape, y.shape

((5000, 400), (5000, 1))

接下来为y标签进行one-hot编码，将10个类标签转换为长度为10的向量

In [4]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

C:\Users\youthPaul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(5000, 10)

In [5]:
y_onehot[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

阅读PDF可以得知，我们可以在神经网络的隐藏层中维护25个激活单元。根据我们的数据集维度可以得知，神经网络的输入单元有400个，输出单元有10个

<img src="image/1.png" width="700" height="550">

首先，我们来实现**前向传播**和**代价函数**

这里需要注意的是，输入层一共有401个激活单元（400个原始输入和1个偏差项），隐藏层一共有26个激活单元

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def forwardPropagation(X, theta1, theta2):
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

代价函数

$J(\Theta) = -\frac{1}{m}\sum\limits_{i=1}^{m} \sum\limits_{k = 1}^{K}{\Big[{{y}_{k}^{(i)}}\log \left( {h_\theta}\left( {{x}^{(i)}} \right) \right)_{k}+\left( 1-{{y}_{k}^{(i)}} \right)\log \left( 1-{h_\theta}\left( {{x}^{(i)}} \right) \right)_{k} \Big]}$

注意此处并没有加入正则化

In [7]:
def cost(params, input_size, hidden_size, output_size, X, y, lamb):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array to parameter matrix for each layer
    theta1 = np.matrix(np.reshape(params[: hidden_size * (input_size + 1)], (hidden_size, input_size + 1)))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) :], (output_size, hidden_size + 1)))

    # run the forward propagation
    a1, z2, a2, z3, h = forwardPropagation(X, theta1, theta2)

    # compute the cost
    J = 0
    for i in range(m):
        first = np.multiply(y[i, :], np.log(h[i, :]))
        second = np.multiply(1 - y[i, :], np.log(1 - h[i, :]))
        J += np.sum(first + second)
    J /= -m

    return J

初始化各参数

In [8]:
input_size = 400
hidden_size = 25
output_size = 10
lamb = 1

# random initialize
params = (np.random.random(size=hidden_size * (input_size + 1) + output_size * (hidden_size + 1)) - 0.5) * 0.25

X = np.matrix(X)
y = np.matrix(y)

# reshape the parameter array to parameter matrix for each layer
theta1 = np.matrix(np.reshape(params[: hidden_size * (input_size + 1)], (hidden_size, input_size + 1)))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) :], (output_size, hidden_size + 1)))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [9]:
a1, z2, a2, z3, h = forwardPropagation(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

计算一下误差

In [10]:
cost(params, input_size, hidden_size, output_size, X, y_onehot, lamb)

7.093594810735645

## 正则化代价函数

<img src="image/2.png" width="900" height="215">

In [11]:
def cost(params, input_size, hidden_size, output_size, X, y, lamb):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array to parameter matrix for each layer
    theta1 = np.matrix(np.reshape(params[: hidden_size * (input_size + 1)], (hidden_size, input_size + 1)))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) :], (output_size, hidden_size + 1)))

    # run the forward propagation
    a1, z2, a2, z3, h = forwardPropagation(X, theta1, theta2)

    # compute the cost
    J = 0
    for i in range(m):
        first = np.multiply(y[i, :], np.log(h[i, :]))
        second = np.multiply(1 - y[i, :], np.log(1 - h[i, :]))
        J += np.sum(first + second)
    J /= -m

    # regularize
    J += lamb / (2 * m) * (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2)))

    return J

In [12]:
cost(params, input_size, hidden_size, output_size, X, y_onehot, lamb)

7.098852378268576

接下来我们实现**反向传播**，我们知道，输出层的误差 $\delta^{(3)} = h - y$

而隐藏层的误差 $\delta^{(2)} = (\Theta^{(2)})^T \delta^{(3)} * g^{\prime}\big( z^{(2)} \big)$

由于sigmoid的特殊性，$g^{\prime}(z) = g(z) * (1 - g(z))$，所以我们可以将其导数编写为：

In [13]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

接下来我们利用**反向传播**来计算梯度

<img src="image/3.png" width="700" height="330">

可以将计算向量化为：$\Delta^{(l)} := \Delta^{(l)} + \delta^{(l + 1)} (a^{(l)})^T$

最后我们有：

$D_{ij}^{(l)} := \frac{1}{m}\Delta_{ij}^{(l)} + \frac{\lambda}{m}\Theta_{ij}^{(l)} \quad if\; j \neq 0$

$D_{ij}^{(l)} := \frac{1}{m}\Delta_{ij}^{(l)} \hspace{40pt} if\; j = 0$

In [14]:
def backwardPropagation(params, input_size, hidden_size, output_size, X, y, lamb):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array to parameter matrix for each layer
    theta1 = np.matrix(np.reshape(params[: hidden_size * (input_size + 1)], (hidden_size, input_size + 1)))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1) :], (output_size, hidden_size + 1)))

    # run the forward propagation
    a1, z2, a2, z3, h = forwardPropagation(X, theta1, theta2)

    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)

    # compute the cost
    for i in range(m):
        first = np.multiply(y[i, :], np.log(h[i, :]))
        second = np.multiply(1 - y[i, :], np.log(1 - h[i, :]))
        J += np.sum(first + second)
    J /= -m

    # regularize
    J += lamb / (2 * m) * (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2)))

    # backprop
    for i in range(m):
        yt = y[i, :] # (1, 10)
        ht = h[i, :] # (1, 10)
        z2t = z2[i, :] # (1, 25)
        a2t = a2[i, :] # (1, 26)
        a1t = a1[i, :] # (1, 401)

        d3 = ht - yt # (1, 10)
        d2 = np.multiply((theta2.T * d3.T).T[:, 1:], sigmoid_gradient(z2t)) # (1, 25)

        delta2 += d3.T * a2t
        delta1 += d2.T * a1t

    delta1 /= m
    delta2 /= m

    # regularize
    delta1[:, 1:] = delta1[:, 1:] + (theta1[:, 1:] * lamb) / m
    delta2[:, 1:] = delta2[:, 1:] + (theta2[:, 1:] * lamb) / m

    # unravel the gradient matrices to a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

尝试运行，看看结果

In [15]:
J, grad = backwardPropagation(params, input_size, hidden_size, output_size, X, y_onehot, lamb)
J, grad.shape

(7.098852378268576, (10285,))

最后，我们进行梯度下降。由于目标函数可能不收敛，我们显式限制迭代次数为250次。

In [16]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backwardPropagation, x0=params, args=(input_size, hidden_size, output_size,
                 X, y_onehot, lamb), method='TNC', jac=True, options={'maxfun': 250})
fmin

 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 0.3356948692482169
       x: [-1.343e+00  1.338e-03 ...  8.923e-01 -9.735e-01]
     nit: 20
     jac: [-3.971e-04  2.677e-07 ... -3.841e-05 -6.471e-05]
    nfev: 250

可以看到最终的损失值已经降到0.5以下，已经最够优秀

接下来我们利用训练好的神经网络进行预测

In [17]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (output_size, (hidden_size + 1))))

a1, z2, a2, z3, h = forwardPropagation(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=int64)

最后，我们计算准确度

In [19]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = sum(correct) / len(correct)
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 99.26%
